<a href="https://colab.research.google.com/github/noahlim99/MAIMO/blob/main/mAImo_Prototype_M2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pytesseract

import os
from PIL import Image
import pytesseract
import re
from google.colab import drive
from openai import OpenAI

In [3]:
def setup_environment():
    """
    Set up the environment, including Google Drive and necessary installations.
    """
    drive.mount('/content/drive')
    os.system("sudo apt-get update && sudo apt-get install -y tesseract-ocr tesseract-ocr-kor")
    os.system("pip install pytesseract")

In [4]:
# 텍스트 추출 함수
def extract_text_from_images(input_dir, output_dir, lang='kor+eng'):
    """
    Extract text from image files in a directory and save as text files.

    Parameters:
        input_dir (str): Directory containing images.
        output_dir (str): Directory to save extracted text.
        lang (str): OCR language settings (default: 'kor+eng').
    """
    os.makedirs(output_dir, exist_ok=True)

    for img_file in os.listdir(input_dir):
        if img_file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            try:
                img_path = os.path.join(input_dir, img_file)
                output_path = os.path.join(output_dir, os.path.splitext(img_file)[0] + '.txt')

                text = pytesseract.image_to_string(Image.open(img_path), lang=lang)
                filtered_text = re.sub(r'[^\w\s가-힣A-Za-z0-9+\-*/=<>.,]', '', text)

                with open(output_path, 'w', encoding='utf-8') as f:
                    f.write(filtered_text)

                print(f"텍스트 추출 완료: {output_path}")
            except Exception as e:
                print(f"이미지 처리 오류: {img_file}, {e}")

In [5]:
# Open AI (Solar Pro) API 설정
def configure_openai(api_key):
    """
    Configure OpenAI API client.
    """
    return OpenAI(api_key=api_key, base_url="https://api.upstage.ai/v1/solar")


In [6]:
# 과목 분류 함수
def classify_subject(content, subjects, client):
    """
    Classify text content into a subject using OpenAI.

    Parameters:
        content (str): Text to classify.
        subjects (list): List of subject options.
        client (OpenAI): Configured OpenAI client.
    """
    subject_list = ", ".join(subjects)
    prompt = f"""
    Classify the following text into one of these subjects:
    {subject_list}.
    Text: "{content}"
    """
    response = client.chat.completions.create(
        model="solar-pro",
        messages=[{"role": "user", "content": prompt}],
        stream=False,
    )
    return response.choices[0].message.content.strip()

# 요약 및 확장 함수
def summarize_and_expand(content, client):
    """
    Summarize and expand text content using OpenAI.

    Parameters:
        content (str): Text to summarize and expand.
        client (OpenAI): Configured OpenAI client.
    """
    prompt = f"""
    Summarize the following text while keeping only subject-relevant information.
    Add "Additional Information:" with relevant context if applicable.
    Text: "{content}"
    """
    response = client.chat.completions.create(
        model="solar-pro",
        messages=[{"role": "user", "content": prompt}],
        stream=False,
    )
    output = response.choices[0].message.content.strip()
    if "Additional Information:" in output:
        summary, additional = output.split("Additional Information:", 1)
        return summary.strip(), additional.strip()
    return output.strip(), ""

In [7]:
# 메인 실행
def main():
    # 환경 설정
    setup_environment()

    # 사용자 입력
    subjects = input("수강 중인 수업을 입력하세요 (콤마로 구분): ").split(",")
    subjects = [subject.strip() for subject in subjects if subject.strip()]

    if not subjects:
        print("수업 목록이 비어 있습니다. 프로그램을 종료합니다.")
        return

    print(f"입력된 수업 목록: {subjects}")

    # 디렉토리 설정
    input_dir = '/content/drive/My Drive/AIDS_Project/Images'
    output_dir = '/content/drive/My Drive/AIDS_Project/texts'

    # 텍스트 추출
    extract_text_from_images(input_dir, output_dir)

    # OpenAI 설정
    client = configure_openai(api_key="up_QlPWg5KR8ix0jWZnOBqsFjSAyltL1")

    # 텍스트 분류 및 요약
    for txt_file in os.listdir(output_dir):
        if txt_file.endswith('.txt'):
            file_path = os.path.join(output_dir, txt_file)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()

            subject = classify_subject(content, subjects, client)
            summary, additional_info = summarize_and_expand(content, client)

            print(f"파일: {txt_file}")
            print(f"과목: {subject}")
            print(f"요약: {summary}")
            if additional_info:
                print(f"추가 정보: {additional_info}")
            print("-" * 50)

if __name__ == "__main__":
    main()

Mounted at /content/drive
수강 중인 수업을 입력하세요 (콤마로 구분): 금융경제학, 수리통계학, 일반생물학, 교육학개론
입력된 수업 목록: ['금융경제학', '수리통계학', '일반생물학', '교육학개론']
텍스트 추출 완료: /content/drive/My Drive/AIDS_Project/texts/IMG_7299.txt
파일: IMG_7299.txt
과목: The given text is related to the subject of "수리통계학" (Statistical Analysis). This is because the text discusses topics related to linear algebra, vector spaces, and matrices, which are fundamental concepts in statistical analysis.

Classification: 수리통계학
요약: Summary: The text discusses the proof that a square matrix M is a projection matrix if it satisfies M^2 = M. It then goes on to explain that if M is a projection matrix, its column space is orthogonal to its complement. The complement is defined as the set of vectors that are orthogonal to all vectors in the column space. The text also mentions a specific example of a projection matrix, which is a generalization of a symmetric orthogonal projection matrix.
추가 정보: The text is written in Korean and contains mathematical symb